<h1 style="color: #4CAF50">Approach 1</h1>

<h2 style="color: #2196F3;">Framework</h2>

<h4 style="color:rgb(241, 234, 231);">Step 1</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">Cargar datos de una base con info estructurada y no estructurada</li>
    <li style="color:rgb(241, 234, 231);">GPT - Leer base y entenderla, hacer preguntas cuando no se entiende qué es cada columna o valor</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 2</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Con las definiciones terminadas crear prompt final con definiciones</li>
    <li style="color:rgb(241, 234, 231);">GPT - Prompt Beautifier con buenas prácticas</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 3</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Línea por línea asignar un cluster de variables, simplificar cada fila (corrida single)</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 4</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Agrupar todas las líneas en clusters con explicaciones</li>
</ul>

In [18]:
import pandas as pd
import json
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os
from tqdm import tqdm
from utils import *
from prompts import *
from tools import *
import pandas as pd
load_dotenv()

api_key = os.getenv('OpenAI_GPT')
client = OpenAI(api_key=api_key)

In [19]:
df = pd.read_csv('250411 - Base Convs.csv')
df.head(3)

,conversation_id,conversation_start,conversation_end,lang,country,user_id,agent_id,shopping_cart_id,context,channel,category,tracking_events,conversation_json
0,b430ab37-15ca-42fb-82ef-eff30addced5,2025-03-19 17:36:54.696,2025-03-19 17:36:56.779,es,MX,NaN,NaN,NaN,"{""despegar_address"":""5215580512459""}",WHATS_APP,NaN,NaN,NaN
1,b4a62f7f-8c88-4699-837a-68cfd7991f8b,2025-03-19 00:37:12.446,2025-03-19 00:38:07.729,NaN,NaN,NaN,NaN,NaN,"{""category_events"":""without_category"",""client_...",IVR,without_category,"['{event_type=category_events, event_name=with...",NaN
2,b523f523-7bc4-4159-819e-2b78863472af,2025-03-19 21:35:52.625,2025-03-19 21:35:52.640,NaN,NaN,NaN,NaN,NaN,{},IVR,NaN,NaN,NaN


In [3]:
df.isnull().sum()/len(df)*100

conversation_id        0.00
conversation_start     0.00
conversation_end       0.00
lang                  10.10
country               10.75
user_id               86.75
agent_id              70.35
shopping_cart_id      86.75
context                0.00
channel                0.00
category               8.40
tracking_events       27.25
conversation_json     89.10
dtype: float64

In [4]:
df.user_id.fillna(0, inplace=True)
df.shopping_cart_id.fillna(0, inplace=True)
df.conversation_json.fillna('No conversation', inplace=True)
df.agent_id.fillna(0, inplace=True)

In [5]:
#df.dropna(subset=['user_id'], inplace=True)
df.isnull().sum()/len(df)*100

conversation_id        0.00
conversation_start     0.00
conversation_end       0.00
lang                  10.10
country               10.75
user_id                0.00
agent_id               0.00
shopping_cart_id       0.00
context                0.00
channel                0.00
category               8.40
tracking_events       27.25
conversation_json      0.00
dtype: float64

In [3]:
### En base al dataframe lo leemos todo y lo describimos, con logicas de negocio y definiciones necesarias
messages = [{"role": "system", "content": system_read_db},
            {"role": "user", "content": "This dataframe is from a customer service interactions database that logs agent-client calls. I need you to describe the 'df' database for me and create the schema"}]

schema = describe_dataframe(df, messages, describe_tools)
print(schema)

Tool Calling:  read_columns
conversation_id, conversation_start, conversation_end, lang, country, user_id, agent_id, shopping_cart_id, context, channel, category, tracking_events, conversation_json
Tool Calling:  read_column_values
Reading column:  conversation_id


KeyboardInterrupt: 

In [20]:
## Con la pregunta del usuario embellecemos el prompt para que sea lo mas entendible posible para la clusterizacion
messages = [{"role": "system", "content": system_prompt_beautifier},
            {"role": "user", "content": "Quiero crear clusters a partir un dataframe sobre interacciones de los clientes de despegar.com"}]

new_prompt = prompt_beautifier(df, messages, retrieve_dataframe)

Tool Calling:  get_dataframe
Tool Calling:  read_columns
conversation_id, conversation_start, conversation_end, lang, country, user_id, agent_id, shopping_cart_id, context, channel, category, tracking_events, conversation_json
Tool Calling:  read_column_values
Reading column:  conversation_id
Tool Calling:  read_column_values
Reading column:  conversation_start
Tool Calling:  read_column_values
Reading column:  conversation_end
Tool Calling:  read_column_values
Reading column:  lang
Tool Calling:  read_column_values
Reading column:  country
Tool Calling:  read_column_values
Reading column:  user_id
Tool Calling:  read_column_values
Reading column:  agent_id
Tool Calling:  read_column_values
Reading column:  shopping_cart_id
Tool Calling:  read_column_values
Reading column:  context
Tool Calling:  read_column_values
Reading column:  channel
Tool Calling:  read_column_values
Reading column:  category
Tool Calling:  read_column_values
Reading column:  tracking_events
Tool Calling:  read_c

In [21]:
json.loads(new_prompt)['prompt']

"Utiliza el siguiente DataFrame para crear clusters basados en las interacciones de los clientes de Despegar.com. El DataFrame contiene las siguientes columnas: 'conversation_id' (identificador único de la conversación), 'conversation_start' (fecha y hora de inicio de la conversación), 'conversation_end' (fecha y hora de finalización de la conversación), 'lang' (idioma de la conversación), 'country' (país del cliente), 'user_id' (identificador único del usuario), 'agent_id' (identificador del agente que atendió la conversación), 'shopping_cart_id' (identificador del carrito de compras asociado a la conversación), 'context' (información contextual de la conversación en formato JSON), 'channel' (canal a través del cual se realizó la conversación), 'category' (categoría de la conversación), 'tracking_events' (eventos de seguimiento relacionados con la conversación en formato de lista JSON), y 'conversation_json' (detalle de la conversación en formato JSON). Considera la lógica de negocio 

In [23]:
print(json.loads(new_prompt)['unstructured_columns'])

[{'column_name': 'context', 'description': 'Información contextual de la conversación en formato JSON.'}, {'column_name': 'tracking_events', 'description': 'Eventos de seguimiento relacionados con la conversación en formato de lista JSON.'}, {'column_name': 'conversation_json', 'description': 'Detalle de la conversación en formato JSON, incluyendo mensajes y participantes.'}]


In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed
# Diccionario para mapear las tipificaciones a los textos

In [10]:

df_processed = df.copy()
columns_list = json.loads(new_prompt)['unstructured_columns']

for element in range(len(columns_list)):
    column_name = columns_list[element]['column_name']
    description = columns_list[element]['description']
    print('Starting with column:', column_name)
    print('Description:', description)
    
        # Función para procesar la conversación
    def procesar_fila(i):
        data = df_processed[column_name][i]

        try:
            # Realizar la llamada a la API de OpenAI
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages = [{"role": "system", "content": sumup_system.format(column_name=column_name, description=description)},
                            {"role": "user", "content": sumup_user.format(data=data)}],
                temperature=0,
                max_tokens=400,
                seed=50
            )

            # Devolver el resultado
            return i, response.choices[0].message.content
        except openai.BadRequestError as e:
            print(f"Error en la fila {i}: {e}")
            return i, "Error"

    # Usar ThreadPoolExecutor para paralelizar
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = {executor.submit(procesar_fila, i): i for i in range(len(df_processed))}
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            i, result = future.result()
            if result is not None:
                df_processed.at[i, column_name] = result
            else:
                # Si el texto es "Texto por defecto", continuar sin asignar resultado
                continue

df_processed

Starting with column: context
Description: This column contains contextual information about the conversation, including tags and events.


100%|██████████| 2000/2000 [03:28<00:00,  9.61it/s]


Starting with column: tracking_events
Description: This column contains a list of events tracked during the conversation.


100%|██████████| 2000/2000 [01:45<00:00, 18.93it/s]


Starting with column: conversation_json
Description: This column contains detailed conversation logs including participant roles and messages.


100%|██████████| 2000/2000 [01:29<00:00, 22.23it/s]


,conversation_id,conversation_start,conversation_end,lang,country,user_id,agent_id,shopping_cart_id,context,channel,category,tracking_events,conversation_json
0,b430ab37-15ca-42fb-82ef-eff30addced5,2025-03-19 17:36:54.696,2025-03-19 17:36:56.779,es,MX,0,0,0.0,"Key-value pair, where the key is ""despegar_add...",WHATS_APP,NaN,Null value.,Null value.
1,b4a62f7f-8c88-4699-837a-68cfd7991f8b,2025-03-19 00:37:12.446,2025-03-19 00:38:07.729,NaN,NaN,0,0,0.0,The data indicates that the conversation falls...,IVR,without_category,Key-value pair indicating an event type catego...,Null value.
2,b523f523-7bc4-4159-819e-2b78863472af,2025-03-19 21:35:52.625,2025-03-19 21:35:52.640,NaN,NaN,0,0,0.0,Null value.,IVR,NaN,Null value.,Null value.
3,b53fe9e6-27ce-4043-bed4-6a5fa492e7a2,2025-03-19 13:37:48.153,2025-03-19 13:37:49.590,pt,BR,0,0,0.0,"Key-value pair, where the key is ""despegar_add...",WHATS_APP,sales,Null value.,Null value.
4,b5b9e643-f94e-4132-bf54-715d8361bd3b,2025-03-19 19:03:23.977,2025-03-19 19:09:21.163,pt,BR,0,ajsantos@viajanet.com.br,0.0,Key-value pair indicating the conversation is ...,IVR,sales,The data consists of an array of tracked event...,Null value.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,ee9d023a-c486-4e6c-8fbc-d128965f7514,2025-03-29 17:12:40.329,2025-03-29 17:15:53.857,es,AR,0,saira.carmona@bestday.com,0.0,Key-value pair indicating the conversation is ...,IVR,sales,The data consists of an array of tracking even...,Null value.
1996,eeb94db8-4647-42dc-8f1c-44e7a7f8feff,2025-03-28 16:15:28.852,2025-03-29 13:14:02.301,pt,BR,0,lucas.pereira.prado@decolar.com,0.0,Key-value pairs indicating the conversation co...,WHATS_APP,sales,Null value.,Null value.
1997,eeefe197-2cbd-4967-bc14-8bbc6fdcd051,2025-03-29 14:45:28.764,2025-03-29 14:45:31.998,es,MX,0,0,0.0,The data indicates that the event falls under ...,IVR,without_category,Key-value pair indicating an event type catego...,Null value.
1998,ef25c457-190a-4f6d-8af3-31203972cff7,2025-03-29 06:41:44.664,2025-03-29 07:28:30.349,es,AR,0,0,0.0,"Key-value pair, where the key is ""despegar_add...",WHATS_APP,aftersale,Key-value pair indicating an information event...,Null value.


In [49]:
### GRAN PARTE DE NULLS, COMO LOS TRATAMOS? LOS BAJAMOS?

In [12]:
import tiktoken

valores = []
for row in range(len(df_processed)):
    valores.append(df_processed.loc[row].to_dict())

print("Cantidad de filas totales: ", len(valores)) #Cantidad total de filas

# Choose encoding for the model you are using
encoding = tiktoken.get_encoding("cl100k_base")

# Convertimos las filas a texto para contar tokens
valores_str = '\n'.join([str(valores[i]) for i in range(len(valores))])
tokens = encoding.encode(valores_str)
print("Cantidad de tokens totales: ",len(tokens)) #Cantidad total de tokens
n_chunks = len(tokens)/64000
print("Cantidad de chunks necesarios: ",round(n_chunks,))

chunk_size = round(len(valores)/n_chunks)
print("Cantidad de filas por chunk: ",chunk_size) #Cantidad de filas por chunk

def chunk_list(data, chunk_size):
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

chunks = chunk_list(valores, chunk_size=round(chunk_size))
print("Chunks totales: ", len(chunks))
# Example: print the first chunk
print(chunks[0])  # This will show the first 3 dicts


Cantidad de filas totales:  2000
Cantidad de tokens totales:  543454
Cantidad de chunks necesarios:  8
Cantidad de filas por chunk:  236
Chunks totales:  9
[{'conversation_id': 'b430ab37-15ca-42fb-82ef-eff30addced5', 'conversation_start': '2025-03-19 17:36:54.696', 'conversation_end': '2025-03-19 17:36:56.779', 'lang': 'es', 'country': 'MX', 'user_id': 0, 'agent_id': 0, 'shopping_cart_id': 0.0, 'context': 'Key-value pair, where the key is "despegar_address" and the value is "5215580512459". This indicates a specific address related to the user or interaction.', 'channel': 'WHATS_APP', 'category': nan, 'tracking_events': 'Null value.', 'conversation_json': 'Null value.'}, {'conversation_id': 'b4a62f7f-8c88-4699-837a-68cfd7991f8b', 'conversation_start': '2025-03-19 00:37:12.446', 'conversation_end': '2025-03-19 00:38:07.729', 'lang': nan, 'country': nan, 'user_id': 0, 'agent_id': 0, 'shopping_cart_id': 0.0, 'context': 'The data indicates that the conversation falls under the "without_cat

In [16]:
system_clusters= """
{beautiful_prompt}

You recieve every row from the dataframe with the values of each column for each row and you have fo find patterns and create clusters based on the values of each column.
Your answer should be the clusters with the name of the cluster, the description of the cluster and the variables involved in the decision.
"""

system_clusters_tot= """
{beautiful_prompt}

Due to the lenght of the dataframe, the clusters were created in chunks of 128000 tokens.
You recieve the initial clusters created for each chunk and you have to create the final clusters based on the initial clusters.
These clusters should include every information from the initial clusters and should be the final clusters.
"""

clusters_user = """
<ROW BY ROW VALUES FOR COLUMNS>

{data}
"""

clusters_tot_user = """
<INITIAL CLUSTERS>

{clusters}
"""

In [17]:
#system_clusters = json.loads(new_prompt)['prompt']
clusters = []
for element in range(len(chunks)):
    # Convert the list of dictionaries to a string representation
    #valores_str = '\n'.join([str(chunks[element]) for i in range(len(chunks[element]))])
    
    # Realizar la llamada a la API de OpenAI
    response = client.chat.completions.create(
        model="gpt-4o",
        messages = [{"role": "system", "content": system_clusters.format(beautiful_prompt=json.loads(new_prompt)['prompt'])},
                    {"role": "user", "content": clusters_user.format(data=chunks[element])}],
        temperature=0,
        max_tokens=1000,
        seed=50
    )
    
    # Imprimir la respuesta
    clusters.append(response.choices[0].message.content)
    print(f"Cluster {element} appended")
    #print(response.choices[0].message.content)


clusters_i = '\n\n-------- NEXT CHUNK -------\n\n'.join(clusters)

# Realizar la llamada a la API de OpenAI
response = client.chat.completions.create(
    model="gpt-4o",
    messages = [{"role": "system", "content": system_clusters_tot.format(beautiful_prompt=json.loads(new_prompt)['prompt'])},
                {"role": "user", "content": clusters_tot_user.format(clusters=clusters_i)}],
    temperature=0,
    max_tokens=2000,
    seed=50
)

# Imprimir la respuesta
clusters.append(response.choices[0].message.content)
print(response.choices[0].message.content)



Cluster 0 appended
Cluster 1 appended
Cluster 2 appended
Cluster 3 appended
Cluster 4 appended
Cluster 5 appended
Cluster 6 appended
Cluster 7 appended
Cluster 8 appended
Based on the initial clusters provided for each chunk, I have synthesized the final clusters by integrating the common themes and unique insights from each chunk. Here are the final clusters:

1. **Regional Language and Country Patterns:**
   - **Description:** Conversations grouped by language and country to identify regional patterns and preferences.
   - **Variables Involved:** `lang`, `country`
   - **Examples:**
     - Spanish (es) in Argentina (AR), Mexico (MX), Colombia (CO), Chile (CL), and Peru (PE).
     - Portuguese (pt) in Brazil (BR).

2. **Channel Preference and Usage:**
   - **Description:** Grouping based on the communication channel to understand preferred methods and their usage.
   - **Variables Involved:** `channel`
   - **Examples:**
     - IVR (Interactive Voice Response) for structured inquiries

In [103]:
df.to_csv('AIO-BC-df_processed.csv', index=False)

<h1 style="color: #4CAF50">Approach 2</h1>

<h2 style="color: #2196F3;">Framework</h2>

<h4 style="color:rgb(241, 234, 231);">Step 1</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">Cargar datos de una base con info estructurada y no estructurada</li>
    <li style="color:rgb(241, 234, 231);">Armar clusters iniciales unicamente con la info estructurada</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 2</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Leer base y entenderla + info no estructurada, hacer preguntas cuando no se entiende qué es cada columna o valor</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 3</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Con las definiciones terminadas crear prompt final con definiciones</li>
    <li style="color:rgb(241, 234, 231);">GPT - Prompt Beautifier con buenas prácticas</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 4</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Por cada cluster inicial correr en masa las variables para sacar nuevos clusters que se realacionen entre ellos</li>
</ul>

<h4 style="color:rgb(241, 234, 231);">Step 5</h4>
<ul style="list-style-type: square; padding-left: 20px;">
    <li style="color:rgb(241, 234, 231);">GPT - Con todos los clusters finales armar set final de clusters</li>
</ul>